In [1]:
# imports
import pandas as pd
import numpy as np

from datetime import datetime, timezone
import time

import csv, os, joblib

from binance.client import Client
from binance.enums import KLINE_INTERVAL_3MINUTE, SIDE_BUY, SIDE_SELL, FUTURE_ORDER_TYPE_MARKET, FUTURE_ORDER_TYPE_LIMIT, FUTURE_ORDER_TYPE_STOP_MARKET
from binance.exceptions import BinanceAPIException

In [2]:
keys = np.load('keys.npy')
API_Key = keys[0]
Secret_Key = keys[1]

In [3]:
#  client
client = Client(api_key=API_Key, api_secret=Secret_Key)

In [4]:
# Convert ISO 8601 date strings to Unix timestamp (milliseconds)
def iso_to_unix(iso_str):
    dt = datetime.strptime(iso_str, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc)
    return int(dt.timestamp() * 1000)  # Convert to milliseconds

def unix_to_iso(unix_timestamp_ms):
    # Convert milliseconds to seconds
    unix_timestamp_s = unix_timestamp_ms / 1000
    # Create a datetime object from the Unix timestamp
    dt = datetime.utcfromtimestamp(unix_timestamp_s)
    # Format the datetime object as an ISO 8601 date string
    iso_str = dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    return iso_str

In [5]:
def log_data(data, filename):
    """
    Appends the given data to a CSV file.
    Creates a 'trade_log' directory if it doesn't exist.

    :param data: List of data to log.
    :param filename: Name of the file to which the data will be logged.
    """
    os.makedirs('trade_log', exist_ok=True)
    file_path = f'trade_log/{filename}.csv'

    try:
        with open(file_path, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
            # Adding a timestamp to each log entry
            log_entry = [datetime.now().strftime("%Y-%m-%d %H:%M:%S")] + data
            csvwriter.writerow(log_entry)
    except IOError as e:
        print(f"IOError while writing to {file_path}: {e}")

# Example usage
# log(["This is a log message"], "example_log")

In [6]:
def get_usdt_balance(client):
    try:
        # Fetch Futures account information
        futures_account_info = client.futures_account_balance()

        # Search for USDT balance in the Futures account balances
        usdt_balance = next((item for item in futures_account_info if item["asset"] == "USDT"), None)

        if usdt_balance:
            return float(usdt_balance['balance'])  # Returns total USDT balance in Futures account
        else:
            return 0.0  # Returns 0 if USDT is not found in the Futures account
    
    except BinanceAPIException as e:
        return f"An error occurred: {e}"

# balance = get_usdt_balance(client)
# print(f"USDT Balance: {balance}")

In [7]:
def how_much_amount(balance, current_price, leverage):
    amount = (balance/current_price)*leverage//0.001*0.001
    return amount

In [8]:
def get_current_price(client, symbol):
    try:
        # Fetch the current price for the specified symbol
        current_price = client.get_symbol_ticker(symbol=symbol)

        return float(current_price['price'])

    except BinanceAPIException as e:
        return f"An error occurred: {e}"

# # Use the function
# symbol = "BTCUSDT"  # Replace with the symbol you want the price for
# price = get_current_price(client,symbol)
# print(f"Current Price of {symbol}: {price}")

In [9]:
def set_leverage(client, symbol, leverage):
    try:
        # Set the leverage for the specified symbol
        response = client.futures_change_leverage(symbol=symbol, leverage=leverage)
        return response
    except BinanceAPIException as e:
        # Handle potential exceptions from the Binance API
        return {"error": str(e)}

# # Use the function
# symbol = "BTCUSDT"  # Replace with the symbol you want to set leverage for
# leverage = 13  # Replace with the desired leverage value

# response = set_leverage(client, symbol, leverage)
# print(f"Leverage Set for {symbol}: {response}")

In [10]:
def set_margin_type_to_isolated(client, symbol):
    try:
        # Change margin type to 'ISOLATED' for the specified symbol
        response = client.futures_change_margin_type(symbol=symbol, marginType='ISOLATED')
        return response
    except BinanceAPIException as e:
        return f"An error occurred: {e}"

# # Use the function
# symbol = "BTCUSDT"  # Replace with the symbol for which you want to change the margin type

# response = set_margin_type_to_isolated(client, symbol)
# print(response)

In [11]:
def execute_trade(client, symbol, quantity, side, order_type, price=None):
    try:
        # Convert side from 'long':+1/'short':-1 to 'BUY'/'SELL'
        if side == 1:
            trade_side = SIDE_BUY
        elif side == -1:
            trade_side = SIDE_SELL
        else:
            raise ValueError("Invalid side, choose 1 for 'long' or -1 for 'short'")
        
        # Set the order type and add price if necessary
        if order_type.lower() == 'market':
            trade_type = FUTURE_ORDER_TYPE_MARKET
            order_params = {
                'symbol': symbol,
                'side': trade_side,
                'type': trade_type,
                'quantity': quantity
            }
        elif order_type.lower() in ['limit', 'stop_market']:
            if not price:
                raise ValueError("Price must be provided for limit and stop market orders")
            trade_type = FUTURE_ORDER_TYPE_LIMIT if order_type.lower() == 'limit' else FUTURE_ORDER_TYPE_STOP_MARKET
            order_params = {
                'symbol': symbol,
                'side': trade_side,
                'type': trade_type,
                'timeInForce': 'GTC',  # Good till cancelled
                'quantity': quantity,
                'price': price
            }
        else:
            raise ValueError("Invalid order type, choose 'market', 'limit', or 'stop_market'")

        # Create order
        order = client.futures_create_order(**order_params)
        return order

    except BinanceAPIException as e:
        return f"An error occurred: {e}"

# # Example for a limit order
# order = execute_trade(client, "BTCUSDT", 0.01, 1, 'limit', price='20000')
# print(order)

In [12]:
def get_current_position_amount(client, symbol):
    try:
        # Fetch current positions
        positions = client.futures_account()['positions']

        # Find the position for the specified symbol
        position_for_symbol = next((position for position in positions if position['symbol'] == symbol), None)

        if position_for_symbol:
            # return position_for_symbol
            return float(position_for_symbol['positionAmt'])
        else:
            return 0.0  # Returns 0 if no position found for the symbol
    
    except BinanceAPIException as e:
        return f"An error occurred: {e}"

# # Usage example
# symbol = "BTCUSDT"  # Replace with your desired symbol
# position_amount = get_current_position_amount(client, symbol)
# print(f"Current Position Amount for {symbol}: {position_amount}")

In [13]:
def get_current_position(client, symbol):
    try:
        # Fetch current positions
        positions = client.futures_account()['positions']

        # Find the position for the specified symbol
        position_for_symbol = next((position for position in positions if position['symbol'] == symbol), None)

        if position_for_symbol:
            return position_for_symbol
            # return float(position_for_symbol['positionAmt'])
        else:
            return 0.0  # Returns 0 if no position found for the symbol
    
    except BinanceAPIException as e:
        return f"An error occurred: {e}"

# current_position = get_current_position(client, symbol)
# initialMargin = float(current_position['initialMargin'])
# unrealizedProfit = float(current_position['unrealizedProfit'])
# entryPrice = float(current_position['entryPrice'])
# positionAmt = float(current_position['positionAmt'])



# # Usage example
# symbol = "BTCUSDT"  # Replace with your desired symbol
# position_amount = get_current_position_amount(client, symbol)
# print(f"Current Position Amount for {symbol}: {position_amount}")
# {'symbol': 'BTCUSDT', 'initialMargin': '0', 'maintMargin': '0', 'unrealizedProfit': '0.00000000', 'positionInitialMargin': '0', 'openOrderInitialMargin': '0', 'leverage': '125', 'isolated': True, 'entryPrice': '0.0', 'breakEvenPrice': '0.0', 'maxNotional': '50000', 'positionSide': 'BOTH', 'positionAmt': '0.000', 'notional': '0', 'isolatedWallet': '0', 'updateTime': 1699632340026, 'bidNotional': '0', 'askNotional': '0'}

In [14]:
def get_current_futures_open_orders(client, symbol=None):
    try:
        # Fetch open futures orders
        if symbol:
            open_orders = client.futures_get_open_orders(symbol=symbol)
        else:
            # Fetch all open futures orders if no symbol is specified
            open_orders = client.futures_get_open_orders()

        return open_orders
    
    except BinanceAPIException as e:
        return f"An error occurred: {e}"

# Usage example
symbol = "BTCUSDT"  # Replace with your desired symbol, or remove it to fetch all open orders

# open_orders = get_current_futures_open_orders(client, symbol)
# print(open_orders[0]['type'], open_orders[0]['price'])
# print(open_orders[1])

# outputs
# {'orderId': 209224563313, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'ios_o9dT58txamZSfoaM4BZ0', 'price': '0', 'avgPrice': '0', 'origQty': '0.017', 'executedQty': '0', 'cumQuote': '0.00000', 'timeInForce': 'GTE_GTC', 'type': 'STOP_MARKET', 'reduceOnly': True, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '37180', 'workingType': 'MARK_PRICE', 'priceProtect': True, 'origType': 'STOP_MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'NONE', 'goodTillDate': 0, 'time': 1699631599625, 'updateTime': 1699631599625}
# {'orderId': 209224602078, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'ios_vaDyqgPeCxzvQteL6u1w', 'price': '36900', 'avgPrice': '0', 'origQty': '0.017', 'executedQty': '0', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': True, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'LIMIT', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'NONE', 'goodTillDate': 0, 'time': 1699631606575, 'updateTime': 1699631606575}

In [15]:
def cancel_futures_order(client, symbol, order_id):
    try:
        # Cancel the specified futures order
        result = client.futures_cancel_order(symbol=symbol, orderId=order_id)
        return result
    except BinanceAPIException as e:
        return f"An error occurred: {e}"

# # Example usage
# symbol = "BTCUSDT"  # Replace with the symbol of the futures contract
# order_id = 12345678  # Replace with the order ID of the order you want to cancel

# cancel_result = cancel_futures_order(client, symbol, order_id)
# print(cancel_result)

In [16]:
def cancel_all_futures_orders(client, symbol):
    try:
        # Cancel all open futures orders for the specified symbol
        result = client.futures_cancel_all_open_orders(symbol=symbol)
        return result
    except BinanceAPIException as e:
        return f"An error occurred: {e}"

# # Example usage
# symbol = "BTCUSDT"  # Replace with the symbol of the futures contract

# cancel_result = cancel_all_futures_orders(client, symbol)
# print(cancel_result)

In [17]:
def get_current_position_entry_price(client, symbol):
    try:
        # Fetch current positions
        positions = client.futures_account()['positions']

        # Find the position for the specified symbol
        position_for_symbol = next((position for position in positions if position['symbol'] == symbol), None)

        if position_for_symbol:
            return float(position_for_symbol['entryPrice'])
        else:
            return 0.0  # Returns 0 if no position found for the symbol
        
    except BinanceAPIException as e:
        return f"An error occurred: {e}"

# # Usage example
# symbol = "BTCUSDT"  # Replace with your desired symbol
# entry_price = get_current_position_entry_price(client, symbol)
# print(f"Entry Price for {symbol}: {entry_price}")

In [31]:
def fetch_and_prepare_data(client):
    # Fetch historical candle data
    candles = client.get_klines(symbol='BTCUSDT', interval=KLINE_INTERVAL_3MINUTE, limit=321)

    # Create DataFrame
    df = pd.DataFrame(candles, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume',
                                        'Close time', 'Quote asset volume', 'Number of trades',
                                        'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])

    # Convert to numeric and drop unnecessary columns
    for col in ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume', "Number of trades"]:
        df[col] = pd.to_numeric(df[col])
    df.drop(['Open time', 'Close time', 'Ignore'], axis=1, inplace=True)
    # df['Open time'] = df['Open time'].apply(lambda x: unix_to_iso(x))

    # Calculate Moving Averages
    for ma in [5, 10, 20, 30, 60, 120, 240]:
        df[f'MA_{ma}'] = df['Close'].rolling(window=ma).mean()

    # Calculate Bollinger Bands
    df['BB_moving_avg'] = df['Close'].rolling(window=90).mean()
    df['BB_std'] = df['Close'].rolling(window=90).std()
    df['BB_upper_band'] = df['BB_moving_avg'] + (df['BB_std'] * 1)
    df['BB_lower_band'] = df['BB_moving_avg'] - (df['BB_std'] * 1)

    # Drop rows with NaN values
    df.drop(['BB_std'], axis=1, inplace=True)
    df.dropna(inplace=True)

    df = df.tail(80)
    
    df = np.array(df)

    # Return as matrix
    return df

# Use the function
matrix = fetch_and_prepare_data(client)
pd.DataFrame(matrix)

# # print(pd.DataFrame(matrix_data).shape)
# [prediction] = model.predict(matrix)
# predict_max , predict_min = np.array(prediction)
# print(prediction, predict_max , predict_min)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,71078.08,71117.21,71016.2,71051.99,56.3276,4002321.440829,1934,23.85158000,1694559.07430390,71038.586,71108.387,71149.4,71169.795,71258.5435,71323.690667,71542.426,71282.654556,71436.135832,71129.173279
1,71052.0,71110.0,71044.88,71044.88,31.41188,2232503.803579,1454,11.50699000,817863.39533380,71031.92,71095.431,71136.7915,71167.691,71251.012,71318.75775,71539.937958,71281.019778,71436.250974,71125.788581
2,71044.89,71044.89,70986.01,70986.01,48.44565,3440939.061414,1797,13.58228000,964689.40170160,71028.498,71075.639,71124.591,71162.691667,71242.512,71313.463,71536.896333,71277.864222,71436.17773,71119.550714
3,70986.01,71082.79,70986.01,71056.0,45.93225,3262864.093238,2208,32.75816000,2326887.18173310,71043.392,71059.352,71116.9905,71158.691667,71235.488,71308.763,71534.216208,71275.930778,71435.889309,71115.972247
4,71055.99,71099.99,71031.56,71099.99,50.3634,3579192.427606,2233,27.97905000,1988298.05603910,71047.774,71052.803,71109.0955,71156.401667,71230.488,71303.896167,71531.342833,71275.437333,71435.873957,71115.00071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,70610.0,70662.45,70575.25,70656.87,44.4735,3141568.569901,2050,20.37369000,1439128.66092470,70632.556,70637.369,70501.2205,70320.182,70477.469,70817.87575,71173.91375,70684.898,71074.034892,70295.761108
76,70656.87,70656.87,70592.0,70596.3,32.12587,2268786.566736,1434,12.55006000,886290.71844160,70616.874,70643.296,70519.512,70348.725667,70471.518333,70811.36875,71169.109792,70678.237667,71063.645042,70292.830291
77,70596.29,70596.3,70555.56,70555.56,32.41835,2288299.888421,1840,9.72051000,686112.56652110,70597.416,70633.652,70540.7075,70376.578,70468.0845,70804.905417,71164.349625,70671.741444,71054.190402,70289.292487
78,70555.57,70560.0,70500.01,70501.01,37.47506,2643113.599153,1816,11.62056000,819705.89371860,70583.946,70608.177,70557.1015,70397.157333,70463.501167,70798.333333,71159.381417,70665.427778,71045.926235,70284.92932


In [32]:
scaler = joblib.load('Scalers/StandardScaler_80.pkl')

# scale the data

# Assuming data is your 600k matrices concatenated into a single 3D numpy array of shape (600000, 20, 19)
temp = matrix.reshape(-1, 19)  # Reshape to 2D for standardization
temp_normalized = scaler.transform(temp)

# Reshape back to 3D
matrix_normalized = temp_normalized.reshape(-1,19*80)

In [33]:
matrix_normalized.shape

(1, 1520)

In [34]:
model = joblib.load('Models/RFC_model_80.pkl')


In [35]:
prediction = model.predict(matrix_normalized)[0]

[Parallel(n_jobs=28)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=28)]: Done 100 out of 100 | elapsed:    0.0s finished


In [36]:
prediction

0

In [38]:
# variables
symbol = "BTCUSDT"
SL = 10

# import model
try:
    model = joblib.load('Models/RFC_model_80.pkl')
    print('Model Load Success')
except Exception as e:
    print(f"Model loading error: {e}")

# import values
scaler = joblib.load('Scalers/StandardScaler_20.pkl')


def start_trading(client,symbol, leverage,SL):
    try:
        market_condition = fetch_and_prepare_data(client)
        time.sleep(0.125)

        # Scale them for model
        temp = market_condition.reshape(-1, 19)
        temp_normalized = scaler.transform(temp)

        # Reshape back to 2D
        matrix_normalized = temp_normalized.reshape(-1,380)
        
        # prediction
        prediction = model.predict(matrix_normalized)[0]
        
        leverage = leverage
        
        current_position = get_current_position(client,symbol)
        time.sleep(0.125)

        position_amount = float(current_position['positionAmt'])

        # not trading
        if position_amount == 0:

            free_usdt = get_usdt_balance(client)
            time.sleep(0.125)
            current_price = get_current_price(client,symbol)
            time.sleep(0.125)
            amount = how_much_amount(balance = free_usdt, current_price = current_price, leverage=leverage)

            if current_price*amount <5:
                message = 'Not enough money'
                print(message)
                time.sleep(5)

                return market_condition, prediction, message, 0, free_usdt, position_amount, 0, 0, 0
                #return market_condition, prediction, message, direction, balance, balance on trade, entry price, take profit, stop loss
            
        
            if prediction == 0:
                message = 'prediction = 0'
                print(message)
                time.sleep(5)
                
                return market_condition, prediction, message, 0, free_usdt, position_amount, 0, 0, 0
                #return market_condition, prediction, message, direction, balance, balance on trade, entry price, take profit, stop loss

            
            # start trade
            elif prediction != 0: 
                message = 'start trading'
                print(message)
                
                # Determine direction and calculate Take Profit and Stop Loss
                direction = 1 if prediction>0 else -1
                if direction == 1:  # long
                    TakeProfit = round(current_price * 1.01, 1)
                    StopLoss = round(current_price * (1-SL/1000), 1)
                else:  # short
                    TakeProfit = round(current_price * 0.99, 1)
                    StopLoss = round(current_price * (1+SL/1000), 1)

                # Execute the trade
                start_trade = execute_trade(client, symbol, quantity=amount, side=direction, order_type='market')
                time.sleep(0.5)

                current_position = get_current_position(client,symbol)
                time.sleep(0.125)
                entry_price = float(current_position['entryPrice'])
                if entry_price == 0 or type(entry_price) != float:
                    message = 'Error in fetching entry price'
                    print(message)
                    return market_condition, prediction, message, 0, free_usdt, 0, 0, 0, 0
                    #return market_condition, prediction, message, direction, balance, balance on trade, entry price, take profit, stop loss

                # Set TP and SL
                SL_trade = execute_trade(client, symbol, quantity=amount, side=-direction, order_type='stop_market', stop_price=StopLoss)
                time.sleep(0.125)
                TP_trade = execute_trade(client, symbol, quantity=amount, side=-direction, order_type='limit', price=TakeProfit)
                time.sleep(5)

                return market_condition, prediction, message, direction, free_usdt, amount*entry_price, entry_price, TakeProfit, StopLoss
                    #return market_condition, prediction, message, direction, balance, balance on trade, entry price, take profit, stop loss
        
        # we are trading, reset
        elif position_amount != 0:
            
            message = 'no reset'
            print(message)

            return market_condition, prediction, message, direction, 0, amount*entry_price, 0, 0, 0
            #return market_condition, prediction, message, direction, balance, balance on trade, entry price, take profit, stop loss

    except BinanceAPIException as e:
        message = f"An error occurred: {e}"
        return np.zeros((20,19)), -2, message, 0, 0, 0, 0, 0, 0
        #return market_condition, prediction, message, direction, balance, balance on trade, entry price, take profit, stop loss


Model Load Success


In [39]:
def show_current_situation(client, symbol):
    try:
        current_position = get_current_position(client, symbol)
        initialMargin = float(current_position['initialMargin'])
        unrealizedProfit = float(current_position['unrealizedProfit'])
        entryPrice = float(current_position['entryPrice'])
        positionAmt = float(current_position['positionAmt'])

        open_orders = get_current_futures_open_orders(client, symbol)
        if open_orders[0]['type'] == 'LIMIT':
            TakeProfit = float(open_orders[0]['price'])
            StopLoss = float(open_orders[1]['stopPrice'])
        if open_orders[1]['type'] == 'LIMIT':
            TakeProfit = float(open_orders[1]['price'])
            StopLoss = float(open_orders[0]['stopPrice'])

        PNL = unrealizedProfit
        if initialMargin == 0:
            ROE = None
        else:
            ROE = 100*unrealizedProfit/initialMargin
            ROE = round(ROE,2)

        print(PNL, ROE, entryPrice)
        print('limit order at', TakeProfit)
        print('stop-market order at', StopLoss)
        print(' ')
        
    except BinanceAPIException as e:
        return f"An error occurred: {e}"

In [40]:
def close_all_positions_and_cancel_orders(client):
    try:
        # Cancel all open orders across all symbols
        client.futures_cancel_all_open_orders()

        # Fetch current positions
        positions = client.futures_account()['positions']

        # Close each position
        for position in positions:
            symbol = position['symbol']
            amount = float(position['positionAmt'])

            if amount != 0:
                side = 'SELL' if amount > 0 else 'BUY'
                client.futures_create_order(
                    symbol=symbol,
                    side=side,
                    type='MARKET',
                    quantity=abs(amount)
                )
    except BinanceAPIException as e:
        return f"An error occurred: {e}"

# # Example usage
# result = close_all_positions_and_cancel_orders(client)
# print(result)

In [41]:
def close_positions_and_cancel_orders_for_symbol(client, symbol):
    try:
        # Cancel all open orders for the specified symbol
        client.futures_cancel_all_open_orders(symbol=symbol)

        # Fetch current position for the symbol
        position_info = next((position for position in client.futures_account()['positions'] if position['symbol'] == symbol), None)
        
        if position_info:
            amount = float(position_info['positionAmt'])
            if amount != 0:
                side = 'SELL' if amount > 0 else 'BUY'
                client.futures_create_order(
                    symbol=symbol,
                    side=side,
                    type='MARKET',
                    quantity=abs(amount)
                )
    except BinanceAPIException as e:
        return f"An error occurred: {e}"

# # Example usage for a specific symbol
# symbol = "BTCUSDT"  # Replace with the symbol you want to close positions for
# result = close_positions_and_cancel_orders_for_symbol(client, symbol)
# print(result)

In [42]:
def get_binance_server_time_unix(client):
    try:
        # Fetch server time
        server_time = client.get_server_time()
        # Extract the Unix timestamp
        unix_time = server_time['serverTime']
        return unix_time
    except BinanceAPIException as e:
        return f"An error occurred: {e}"


In [43]:
def prepare_log_data(output):
    market_condition, prediction, message, direction, balance, balance_on_trade, entry_price, take_profit, stop_loss = output
    market_condition_flat = np.reshape(np.array(market_condition), -1)
    return [message, direction, balance, balance_on_trade, entry_price, take_profit, stop_loss] + list(market_condition_flat) + list(prediction)

In [44]:
def run():
    try:
        model = joblib.load('Models/RFC_model.pkl')
        print('Model Load Success')
    except Exception as e:
        print(f"Model loading error: {e}")
        log_data([f"Model loading error: {e}"], 'error_log')
        return  # Exit if the model cannot be loaded
    
    while True:
        try:
            server_time = get_binance_server_time_unix(client)
            
            # every 3 minutes, at 2min 55sec
            if (server_time - 1698796800000) % (3 * 60 * 1000) > (2 * 60 + 55) * 1000:
                output = start_trading(client, symbol)
                log_data(prepare_log_data(output), 'trade_log')
                
                
            # every 2 seconds, always
            else:
                time.sleep(2)

        except Exception as e:
            print(f"An error occurred: {e}")
            log_data([f"Error: {e}"], 'error_log')
            time.sleep(10)